# Pre Work

## Importing libraries

In [26]:
# libraries that deal with grabbing from the .env file
from dotenv import load_dotenv
import os

# make url requests
import requests

# make DFs
import pandas as pd

# show images in notebook
from IPython.display import display, Image

## Getting the API key from the .env file

In [27]:

# Remove the cached environment variable (if any)
os.environ.pop('riot_api_key', None)

# Reload the .env file
load_dotenv()

# Fetch the API key
api_key = os.environ.get('riot_api_key')



# Inputs

## Summoner ID and Region inputs

In [42]:
# input username, region code here, asking for user input

# Prompt user for input
gameName = input("Enter your summoner: ")


# Prompt user for input with a default value
tagLine = input("Enter tagline (default: NA1): ")

# Use default value if input is empty
tagLine = tagLine if tagLine.strip() else "NA1"

# Prompt user for input with a default value
region = input("Enter Region: americas, asia, and europe (default: americas): ")

# Use default value if input is empty
region = region if region.strip() else "americas"

print(gameName, tagLine, region)


HGILLIS NA1 americas


# Creating Functions

## get puuid from gameName and tagLine

In [29]:
def get_puuid(summonerId=None, gameName=None, tagLine=None, region='americas'):
    """Gets the puuid from a summonerId or Riot_id and riot_tag
    
    Args:
        summonerId (str, optional): Summoner ID. Defaults to None.
        gameName (str, optional): Riot ID. Defaults to None.
        tagLine (str, optional): Riot Tag. Defaults to None.
        region (str, optional): Region. Defaults to 'americas'.
        
    Returns:
        str: puuid
    """
# using summonerID 
    if summonerId is not None:
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = 'lol/summoner/v4/summoners/'
        print(root_url+endpoint+summonerId+'?api_key='+api_key)
        response = requests.get(root_url+endpoint+summonerId+'?api_key='+api_key)

        return response.json()['puuid']
    else:
        #using gamename + tagline for example HGILLIS + NA1
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = f'riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}'

        response = requests.get(root_url+endpoint+'?api_key='+api_key)

        return response.json()['puuid']
    


## get gameName and tagLine from puuid

In [30]:

#use the players puuid to get their name and region code
def get_name_and_tag(puuid=None, api_key=None):
    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}'

    response = requests.get(link)

    gameName = response.json()['gameName']
    tagLine = response.json()['tagLine']

    return f'{gameName}#{tagLine}'
# this code will output like this: 'SummonerName#1234' for use in other functions

In [31]:
def get_idtag_from_puuid(puuid=None):
    """Gets the riot_id and riot_tag from a puuid
    
    Args:
        puuid (str, optional): puuid. Defaults to None.
        
    Returns:
        id (dict): Dictionary with riot_id and riot_tag.
    """
    root_url = 'https://americas.api.riotgames.com/'
    endpoint = 'riot/account/v1/accounts/by-puuid/'

    response = requests.get(root_url+endpoint+puuid+'?api_key='+api_key)

    id = {
        'gameName': response.json()['gameName'],
        'tagLine': response.json()['tagLine']
    }

    return id
# this code will output like this: {'gameName': 'SummonerName', 'tagLine': '1234'} for further processing

In [32]:
#test get puuid function
get_puuid(gameName=gameName, tagLine=tagLine)

'V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoDgxwUtdIeVTZrmOzjeVHklBRlCJr1tz1g'

In [34]:
#test id function
get_idtag_from_puuid(get_puuid(gameName=gameName, tagLine=tagLine))

{'gameName': 'HGILLIS', 'tagLine': 'NA1'}

In [35]:
def get_match_history(region=None, puuid=None, start=0, count=20):
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/by-puuid/{puuid}/ids'
    query_params = f'?start={start}&count={count}' #changing starting index (start) and number of matches retrieved (count)

    response = requests.get(root_url + endpoint + query_params + '&api_key=' + api_key)
    
    return response.json()

def get_match_data_from_id(region = None, matchId = None):

    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/{matchId}'


    response = requests.get(root_url + endpoint + '?api_key=' + api_key)

    return response.json()

In [44]:
#test get_match_history
match_history = get_match_history(region=region, puuid=get_puuid(gameName=gameName, tagLine=tagLine), start=0, count=5)

In [45]:
#test match data function. Use this url, and the middle option to understand output: https://developer.riotgames.com/apis#match-v5/GET_getMatch
game = get_match_data_from_id(region='americas', matchId=match_history[0])

In [47]:


def process_match_json(match_json, puuid):
    ## Architecture
    metadata = match_json['metadata']
    info = match_json['info']
    players = info['participants']
    participants = metadata['participants']
    teams = info['teams']
    player = players[participants.index(puuid)]
    perks = player['perks']
    stats = perks ['statPerks']
    styles = perks['styles']

    primary = styles[0]
    secondary = styles[1]

    match_id = metadata['matchId']


    game_creation = info['gameCreation']
    game_duration = info['gameDuration']
    game_start_timestamp = info['gameStartTimestamp']
    game_end_timestamp = info['gameEndTimestamp']
    patch = info['gameVersion']

    riot_id = player['riotIdGameName']
    riot_tag = player['riotIdTagline']
    summoner_id = player['summonerId']
    summoner_name = player['summonerName']

    win = player['win']

    champ_id = player['championId']
    champ_transform = player['championTransform']
    champ_level = player['champLevel']

    gold_earned = player['goldEarned']
    neutral_minions_killed = player['neutralMinionsKilled']
    total_minions_killed = player['totalMinionsKilled']

    kills = player['kills']
    deaths = player['deaths']
    assists = player['assists']
    first_blood = player['firstBloodKill']

    total_damage_dealt= player['totalDamageDealtToChampions']
    total_damage_shielded = player['totalDamageShieldedOnTeammates']
    total_damage_taken = player['totalDamageTaken']
    total_damage_healed = player['totalHealsOnTeammates']
    total_minions_killed = player['totalMinionsKilled']
    total_time_cc_dealt = player['totalTimeCCDealt']

    early_surrender = player['gameEndedInEarlySurrender']
    surrender = player['gameEndedInSurrender']

    item0 = player['item0']
    item1 = player['item1']
    item2 = player['item2']
    item3 = player['item3']
    item4 = player['item4']
    item5 = player['item5']
    item6 = player['item6']

    summoner_1_id = player['summoner1Id']
    summoner_2_id = player['summoner2Id']


    wards_placed = player['wardsPlaced']
    vision_score = player['visionScore']







    defense = stats['defense']
    flex = stats['flex']
    offense = stats['offense']


    primary_style = primary['style']
    secondary_style = primary['style']

    primary_keystone = primary['selections'][0]['perk']
    primary_perk_1 = primary['selections'][1]['perk']
    primary_perk_2 = primary['selections'][2]['perk']
    primary_perk_3 = primary['selections'][3]['perk']

    secondary_perk_1 = secondary['selections'][0]['perk']
    secondary_perk_2 = secondary['selections'][1]['perk']


    objectives_stolen = player['objectivesStolen']
    objectives_stolen_assists = player['objectivesStolenAssists']

    ##what is this?
    detector_wars_placed = player['detectorWardsPlaced']
    vision_wards_bought = player['visionWardsBoughtInGame']
    role = player['role']

    matchDF = pd.DataFrame({
        'match_id': [match_id],
        'participants': [participants],
        'game_creation': [game_creation],
        'game_duration': [game_duration],
        'game_start_timestamp': [game_start_timestamp],
        'game_end_timestamp': [game_end_timestamp],
        'patch': [patch],
        'puuid': [puuid],
        'riot_id': [riot_id],
        'riot_tag': [riot_tag],
        'summoner_id': [summoner_id],
        'summoner_name': [summoner_name],
        'win': [win],
        'champion': [champ_id],
        'champion_transform': [champ_transform],
        'champion_level': [champ_level],
        'kills': [kills],
        'deaths': [deaths],
        'assists': [assists],
        'summoner1_id': [summoner_1_id],
        'summoner2_id': [summoner_2_id],
        'gold_earned': [gold_earned],
        'total_minions_killed': [total_minions_killed],
        'total_neutral_minions_killed': [neutral_minions_killed],
        'early_surrender': [early_surrender],
        'surrender': [surrender],
        'first_blood': [first_blood],
        'objectives_stolen': [objectives_stolen],
        'objectives_stolen_assists': [objectives_stolen_assists],
        'total_damage_dealt_champions': [total_damage_dealt],
        'total_damage_taken': [total_damage_taken],
        'total_damage_shielded_teammates': [total_damage_shielded],
        'total_time_crowd_controlled': [total_time_cc_dealt],
        'vision_score': [vision_score],
        'wards_placed': [wards_placed],
        'item0': [item0],
        'item1': [item1],
        'item2': [item2],
        'item3': [item3],
        'item4': [item4],
        'item5': [item5],
        'item6': [item6],
        'perk_keystone': [primary_keystone],
        'perk_primary_row_1': [primary_perk_1],
        'perk_primary_row_2': [primary_perk_2],
        'perk_primary_row_3': [primary_perk_3],
        'perk_secondary_row_1': [secondary_perk_1],
        'perk_secondary_row_2': [secondary_perk_2],
        'perk_primary_style': [primary_style],
        'perk_secondary_style': [secondary_style],
        'perk_shard_defense': [defense],
        'perk_shard_flex': [flex],
        'perk_shard_offense': [offense],
    })

    return matchDF


In [48]:
process_match_json(game, get_puuid(gameName=gameName, tagLine=tagLine))

,match_id,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,puuid,riot_id,riot_tag,...,perk_primary_row_1,perk_primary_row_2,perk_primary_row_3,perk_secondary_row_1,perk_secondary_row_2,perk_primary_style,perk_secondary_style,perk_shard_defense,perk_shard_flex,perk_shard_offense
0,NA1_5169696506,[C_y_7G8dryEs-wAc7LFctidpDqXTk0GE3p7xYdqolv8dB...,1733025855552,917,1733025873060,1733026790527,14.23.636.9832,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,...,8143,8138,8105,8237,8210,8100,8100,5001,5008,5008


In [55]:
match_ids = get_match_history(region='americas', puuid=get_puuid(gameName=gameName, tagLine=tagLine))

df = pd.DataFrame()
for match_id in match_ids:
    game = get_match_data_from_id(region='americas', matchId=match_id)
    matchDF = process_match_json(game, puuid=get_puuid(gameName=gameName, tagLine=tagLine))

    df = pd.concat([df, matchDF])

In [56]:
df

,match_id,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,puuid,riot_id,riot_tag,...,perk_primary_row_1,perk_primary_row_2,perk_primary_row_3,perk_secondary_row_1,perk_secondary_row_2,perk_primary_style,perk_secondary_style,perk_shard_defense,perk_shard_flex,perk_shard_offense
0,NA1_5169696506,[C_y_7G8dryEs-wAc7LFctidpDqXTk0GE3p7xYdqolv8dB...,1733025855552,917,1733025873060,1733026790527,14.23.636.9832,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,...,8143,8138,8105,8237,8210,8100,8100,5001,5008,5008
0,NA1_5168666185,[d66zRs2wr3In3cbz_fPo1cbUt99SyQNUr78FGsGcm_Bma...,1732941578702,1282,1732941697696,1732942979667,14.23.636.9832,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,...,8304,8313,8410,8138,8105,8300,8300,5001,5010,5007
0,NA1_5168623889,[lPH7hgBP0tf_i0ut8KcVBRcB55vADrptwIf9jDiK5noh-...,1732939477237,1770,1732939503114,1732941273432,14.23.636.9832,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,...,8226,8234,8236,8345,8347,8200,8200,5001,5008,5005
0,NA1_5168583406,[qQP3i8fzkUkw5tnG_Pi013-C94iufMjC6Y5bFi7v7mfHL...,1732937542983,1546,1732937622502,1732939169036,14.23.636.9832,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,...,8463,8429,8451,9105,9111,8400,8400,5001,5001,5007
0,NA1_5168547222,[Dki_ODK5HJ2I5Hi-IY9D6ciZM994MnoEIiy1-l5tB9l8W...,1732935880241,1480,1732935913437,1732937394161,14.23.636.9832,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,...,8143,8136,8135,8347,8306,8100,8100,5001,5008,5008
0,NA1_5168495833,[BEWmOX1eM-jtC-Fl0YMDFZaTrcZfO906fsH_9l6J0f0dN...,1732933125942,2281,1732933154632,1732935435844,14.23.636.9832,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,...,8275,8210,8232,8138,8105,8200,8200,5001,5008,5007
0,NA1_5168448938,[v9cKAiyHdtPQ4HXfm7RKoUiH4Qme9tpon2h-j6977gySZ...,1732930638020,1882,1732930667438,1732932549963,14.23.636.9832,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,...,8009,9104,8017,8410,8345,8000,8000,5011,5008,5005
0,NA1_5168410364,[w9MPFQzlXX2JRTUwWvByGmnorl-kgmNjY8BfMQ4Lppfg-...,1732928571135,1711,1732928681998,1732930393697,14.23.636.9832,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,...,8143,8138,8135,8210,8232,8100,8100,5001,5008,5008
0,NA1_5168372791,[V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRo...,1732926216349,1697,1732926236925,1732927934006,14.23.636.9832,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,...,8446,8444,8242,8345,8347,8400,8400,5001,5008,5005
0,NA1_5166551901,[6j4_7s4rN4IThsT2pa4sgnviYG-kE2TsWm5XOsoYEMMjy...,1732766627578,3618,1732766703849,1732770322132,14.23.636.9832,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,...,8143,8138,8135,9104,9111,8100,8100,5001,5008,5005


In [57]:
perk = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perks.json'
perk_styles = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perkstyles.json'
champ = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champion-summary.json'
item = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/items.json'

perk_json = requests.get(perk).json()
perk_styles_json = requests.get(perk_styles).json()
champ_json = requests.get(champ).json()
item_json = requests.get(item).json()

In [58]:
def json_extract(obj, key):
    arr = []
    
    def extract(obj, arr, key):
        if isinstance(obj, dict):
            for k, v in obj.items():
                if k == key:
                    arr.append(v)
                elif isinstance(v, (dict, list)):
                    extract(v, arr, key)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values


In [59]:
# Base URL for the champion icons
base_url = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champion-icons/'

# Add a new column with the image URLs
df['image_url'] = df['champion'].apply(lambda x: f"{base_url}{x}.png")

# Display the DataFrame
print(df)


         match_id                                       participants  \
0  NA1_5169696506  [C_y_7G8dryEs-wAc7LFctidpDqXTk0GE3p7xYdqolv8dB...   
0  NA1_5168666185  [d66zRs2wr3In3cbz_fPo1cbUt99SyQNUr78FGsGcm_Bma...   
0  NA1_5168623889  [lPH7hgBP0tf_i0ut8KcVBRcB55vADrptwIf9jDiK5noh-...   
0  NA1_5168583406  [qQP3i8fzkUkw5tnG_Pi013-C94iufMjC6Y5bFi7v7mfHL...   
0  NA1_5168547222  [Dki_ODK5HJ2I5Hi-IY9D6ciZM994MnoEIiy1-l5tB9l8W...   
0  NA1_5168495833  [BEWmOX1eM-jtC-Fl0YMDFZaTrcZfO906fsH_9l6J0f0dN...   
0  NA1_5168448938  [v9cKAiyHdtPQ4HXfm7RKoUiH4Qme9tpon2h-j6977gySZ...   
0  NA1_5168410364  [w9MPFQzlXX2JRTUwWvByGmnorl-kgmNjY8BfMQ4Lppfg-...   
0  NA1_5168372791  [V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRo...   
0  NA1_5166551901  [6j4_7s4rN4IThsT2pa4sgnviYG-kE2TsWm5XOsoYEMMjy...   
0  NA1_5166252192  [65b4YhzrAnliRUdeOfuFBYyFEd3QvgnTAhuxTOj4Ocw9i...   
0  NA1_5166212577  [65b4YhzrAnliRUdeOfuFBYyFEd3QvgnTAhuxTOj4Ocw9i...   
0  NA1_5165218688  [BMs-7Uhoyzz3UYlhjVB4X-K4y2qMjDch_yWi30LKf2uq

In [60]:
from IPython.display import display, Image

# Display all champion images from the DataFrame
for image_url in df['image_url']:
    display(Image(url=image_url))


In [61]:
# code to replace all of the perk numbers with the actual perks

perk_ids = json_extract(perk_json, 'id')
perk_names = json_extract(perk_json, 'name')

perk_dict = dict(map(lambda i, j : (int(i),j), perk_ids, perk_names))

perk_styles_ids = json_extract(perk_styles_json, 'id')
perk_styles_names = json_extract(perk_styles_json, 'name')

perk_styles_dict = dict(map(lambda i, j : (int(i),j), perk_styles_ids, perk_styles_names))

In [62]:
# code to replace all of the champion numbers with the actual champions
champ_ids = json_extract(champ_json, 'id')
champ_names = json_extract(champ_json, 'name')

# Assuming champ_json is a list of dictionaries with each dictionary containing 'id' and 'name'
champ_dict = {int(champ['id']): champ['name'] for champ in champ_json}



In [63]:
# code to replace all of the item numbers with the actual items
item_ids = json_extract(item_json, 'id')
item_names = json_extract(item_json, 'name')

# Assuming champ_json is a list of dictionaries with each dictionary containing 'id' and 'name'
item_dict = {int(item['id']): item['name'] for item in item_json}


In [64]:
pd.options.display.max_columns = 100

df.replace(perk_dict).replace(perk_styles_dict)

# Replace only the 'champion' column with the values from champ_dict
df['champion'] = df['champion'].replace(champ_dict)

# replace the item columns with the the actual items names from the item_dict
df['item0'] = df['item0'].replace(item_dict)
df['item1'] = df['item1'].replace(item_dict)
df['item2'] = df['item2'].replace(item_dict)
df['item3'] = df['item3'].replace(item_dict)
df['item4'] = df['item4'].replace(item_dict)
df['item5'] = df['item5'].replace(item_dict)
df['item6'] = df['item6'].replace(item_dict)
df


,match_id,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,puuid,riot_id,riot_tag,summoner_id,summoner_name,win,champion,champion_transform,champion_level,kills,deaths,assists,summoner1_id,summoner2_id,gold_earned,total_minions_killed,total_neutral_minions_killed,early_surrender,surrender,first_blood,objectives_stolen,objectives_stolen_assists,total_damage_dealt_champions,total_damage_taken,total_damage_shielded_teammates,total_time_crowd_controlled,vision_score,wards_placed,item0,item1,item2,item3,item4,item5,item6,perk_keystone,perk_primary_row_1,perk_primary_row_2,perk_primary_row_3,perk_secondary_row_1,perk_secondary_row_2,perk_primary_style,perk_secondary_style,perk_shard_defense,perk_shard_flex,perk_shard_offense,image_url
0,NA1_5169696506,[C_y_7G8dryEs-wAc7LFctidpDqXTk0GE3p7xYdqolv8dB...,1733025855552,917,1733025873060,1733026790527,14.23.636.9832,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,True,Sylas,0,10,2,4,1,14,4,4274,72,0,False,True,False,0,0,7076,13048,0,133,2,2,Sorcerer's Shoes,Hextech Rocketbelt,Refillable Potion,0,0,0,Stealth Ward,8112,8143,8138,8105,8237,8210,8100,8100,5001,5008,5008,https://raw.communitydragon.org/latest/plugins...
0,NA1_5168666185,[d66zRs2wr3In3cbz_fPo1cbUt99SyQNUr78FGsGcm_Bma...,1732941578702,1282,1732941697696,1732942979667,14.23.636.9832,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,False,Twisted Fate,0,10,1,5,14,14,4,6047,10,0,False,True,False,0,0,9397,15121,0,159,17,7,Bloodsong,Refillable Potion,Rapid Firecannon,Aether Wisp,Boots of Swiftness,Bandleglass Mirror,Oracle Lens,8351,8304,8313,8410,8138,8105,8300,8300,5001,5010,5007,https://raw.communitydragon.org/latest/plugins...
0,NA1_5168623889,[lPH7hgBP0tf_i0ut8KcVBRcB55vADrptwIf9jDiK5noh-...,1732939477237,1770,1732939503114,1732941273432,14.23.636.9832,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,False,Twisted Fate,0,14,4,9,19,3,4,9245,24,0,False,False,False,0,0,15015,23112,0,170,16,7,Bloodsong,Shurelya's Battlesong,Boots of Swiftness,Rapid Firecannon,Winged Moonplate,Chain Vest,Oracle Lens,8229,8226,8234,8236,8345,8347,8200,8200,5001,5008,5005,https://raw.communitydragon.org/latest/plugins...
0,NA1_5168583406,[qQP3i8fzkUkw5tnG_Pi013-C94iufMjC6Y5bFi7v7mfHL...,1732937542983,1546,1732937622502,1732939169036,14.23.636.9832,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,False,Galio,0,18,8,17,38,32,4,15753,41,0,False,False,False,0,0,32661,65046,799,218,0,0,Fimbulwinter,Zhonya's Hourglass,Hollow Radiance,Mercury's Treads,Riftmaker,Thornmail,Poro-Snax,8439,8463,8429,8451,9105,9111,8400,8400,5001,5001,5007,https://raw.communitydragon.org/latest/plugins...
0,NA1_5168547222,[Dki_ODK5HJ2I5Hi-IY9D6ciZM994MnoEIiy1-l5tB9l8W...,1732935880241,1480,1732935913437,1732937394161,14.23.636.9832,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,False,Poppy,0,11,4,8,10,14,4,8287,8,0,False,True,False,0,0,14816,18804,0,165,39,15,Bloodsong,Locket of the Iron Solari,Plated Steelcaps,Dead Man's Plate,Ruby Crystal,0,Oracle Lens,9923,8143,8136,8135,8347,8306,8100,8100,5001,5008,5008,https://raw.communitydragon.org/latest/plugins...
0,NA1_5168495833,[BEWmOX1eM-jtC-Fl0YMDFZaTrcZfO906fsH_9l6J0f0dN...,1732933125942,2281,1732933154632,1732935435844,14.23.636.9832,V8lkRF0RP5WLyEAM7kf0F4RJanDfdGveBZESxJBZPxaRoD...,HGILLIS,NA1,yw8eql-EBvf1djPzM__tH1wGBOqqiPaYbtHx3O6pBOO7bZI,HGILLIS,True,Ivern,0,17,1,10,16,11,4,12613,6,188,False,False,False,0,0,14397,30411,9564,230,18,7,0,Redemption,Shurelya's Battlesong,Dawncore,Moonstone Renewer,Synchronized Souls,Stealth Ward,8214,8275,8210,8232,8138,8105,8200,8200,5001,5008,5007,https://raw.communitydragon.org/latest/plugins...
0,NA1_5168448938,[v9c

In [65]:
# Replace the values based on the dictionaries and drop the first 12 columns
filtered_df = df.replace(perk_dict).replace(perk_styles_dict).iloc[:10, 12:]

#drop columns i dont want
filtered_df = filtered_df.drop(columns = ['champion_transform', 'total_neutral_minions_killed', 'early_surrender', 'surrender',  'first_blood', 'objectives_stolen',  'objectives_stolen_assists' ])

# Replace True with 'Win' and False with 'Loss' using map
# Replace True with 'Win' and False with 'Loss'
filtered_df['win'] = filtered_df['win'].replace({True: 'Win', False: 'Loss'})




# Output the result to a CSV file
filtered_df.to_csv('match_hist2.csv', index=False)

filtered_df


,win,champion,champion_level,kills,deaths,assists,summoner1_id,summoner2_id,gold_earned,total_minions_killed,total_damage_dealt_champions,total_damage_taken,total_damage_shielded_teammates,total_time_crowd_controlled,vision_score,wards_placed,item0,item1,item2,item3,item4,item5,item6,perk_keystone,perk_primary_row_1,perk_primary_row_2,perk_primary_row_3,perk_secondary_row_1,perk_secondary_row_2,perk_primary_style,perk_secondary_style,perk_shard_defense,perk_shard_flex,perk_shard_offense,image_url
0,Win,Sylas,10,2,4,1,14,4,4274,72,7076,13048,0,133,2,2,Sorcerer's Shoes,Hextech Rocketbelt,Refillable Potion,0,0,0,Stealth Ward,Electrocute,Sudden Impact,Eyeball Collection,Relentless Hunter,Scorch,Transcendence,Precision,Precision,Health Scaling,Adaptive Force,Adaptive Force,https://raw.communitydragon.org/latest/plugins...
0,Loss,Twisted Fate,10,1,5,14,14,4,6047,10,9397,15121,0,159,17,7,Bloodsong,Refillable Potion,Rapid Firecannon,Aether Wisp,Boots of Swiftness,Bandleglass Mirror,Oracle Lens,Glacial Augment,Magical Footwear,Triple Tonic,Approach Velocity,Eyeball Collection,Relentless Hunter,Inspiration,Inspiration,Health Scaling,Move Speed,Ability Haste,https://raw.communitydragon.org/latest/plugins...
0,Loss,Twisted Fate,14,4,9,19,3,4,9245,24,15015,23112,0,170,16,7,Bloodsong,Shurelya's Battlesong,Boots of Swiftness,Rapid Firecannon,Winged Moonplate,Chain Vest,Oracle Lens,Arcane Comet,Manaflow Band,Celerity,Gathering Storm,Biscuit Delivery,Cosmic Insight,Sorcery,Sorcery,Health Scaling,Adaptive Force,Attack Speed,https://raw.communitydragon.org/latest/plugins...
0,Loss,Galio,18,8,17,38,32,4,15753,41,32661,65046,799,218,0,0,Fimbulwinter,Zhonya's Hourglass,Hollow Radiance,Mercury's Treads,Riftmaker,Thornmail,Poro-Snax,Aftershock,Font of Life,Conditioning,Overgrowth,Legend: Haste,Triumph,Resolve,Resolve,Health Scaling,Health Scaling,Ability Haste,https://raw.communitydragon.org/latest/plugins...
0,Loss,Poppy,11,4,8,10,14,4,8287,8,14816,18804,0,165,39,15,Bloodsong,Locket of the Iron Solari,Plated Steelcaps,Dead Man's Plate,Ruby Crystal,0,Oracle Lens,Hail of Blades,Sudden Impact,Zombie Ward,Treasure Hunter,Cosmic Insight,Hextech Flashtraption,Precision,Precision,Health Scaling,Adaptive Force,Adaptive Force,https://raw.communitydragon.org/latest/plugins...
0,Win,Ivern,17,1,10,16,11,4,12613,6,14397,30411,9564,230,18,7,0,Redemption,Shurelya's Battlesong,Dawncore,Moonstone Renewer,Synchronized Souls,Stealth Ward,Summon Aery,Nimbus Cloak,Transcendence,Waterwalking,Eyeball Collection,Relentless Hunter,Sorcery,Sorcery,Health Scaling,Adaptive Force,Ability Haste,https://raw.communitydragon.org/latest/plugins...
0,Loss,Ashe,15,3,7,2,21,4,10801,216,10956,19201,0,666,11,5,Runaan's Hurricane,Kraken Slayer,Berserker's Greaves,Bloodthirster,0,0,Farsight Alteration,Lethal Tempo,Presence of Mind,Legend: Alacrity,Cut Down,Approach Velocity,Biscuit Delivery,Domination,Domination,Health,Adaptive Force,Attack Speed,https://raw.communitydragon.org/latest/plugins...
0,Loss,Gragas,13,2,10,8,11,4,Glacial Augment,28,12300,32734,0,295,13,5,Gustwalker Hatchling,Shadowflame,Stormsurge,Sorcerer's Shoes,0,0,Oracle Lens,Dark Harvest,Sudden Impact,Eyeball Collection,Treasure Hunter,Transcendence,Waterwalking,Precision,Precision,Health Scaling,Adaptive Force,Adaptive Force,https://raw.communitydragon.org/latest/plugins...
0,Loss,Jax,14,0,6,4,12,4,Cash Back,175,6939,20892,0,65,14,7,Doran's Shield,Trinity Force,Plated Steelcaps,Tunneler,Caulfield's Warhammer,Ruby Crystal,Stealth Ward,Grasp of the Undying,Demolish,Second Wind,Unflinching,Biscuit Delivery,Cosmic Insight,Resolve,Resolve,Health Scaling,Adaptive Force,Attack Speed,https://raw.communitydragon.org/latest/plugins...
0,Win,Graves,18,15,16,10,11,4,23575,64,45724,66027,0,671,32,13,The Collector,Youmuu's Ghostblade,Guardian Angel,Lord Dominik's Regards,Infinity Edge,Immortal Shieldbow,Stealth Ward,Dark Harvest,Sudden Impact,Eyeball Collection,Treasure Hunter,Legend: Alacrity,Triumph,Precision,Precision,Health Scaling,